[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github.com/jkanclerz/analiza-tekstu/blob/master/02_Eksploracyjna_analiza_dokumentow_reprezentacja-wektorowa.ipynb)

# Analiza dokumentów tekstowych	

# Reprezentacja numeryczna

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text

In [1]:
pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  Using cached scikit_learn-1.0.1-cp39-cp39-macosx_10_13_x86_64.whl (8.0 MB)
     |████████████████████████████████| 33.2 MB 260 kB/s            
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Using cached threadpoolctl-3.0.0-py3-none-any.whl (14 kB)
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.feature_extraction.text import  CountVectorizer

## Bag of words
Pozwala reprezentować dane tekstowe jako wektor cech(ang. feature vector). Reprezentacja bag-of-words jest niezwykle prosta, sprowadza się do 2 kroków. 1. Stworzenie słownika unikalnych wyrazów - zbiór wyrazów z całej kolekcji dokumentów 2. Stworzenie reprezentacji wektorowej dla każdego z dokumentów zawierającej częstość wystąpień dla poszczególnych wyrazów

Jako, że pojedyńczy dokument zawiera wyłącznie mały wycinek z całego zbioru dostępnych wyrazów, wektor cech zawiera głównie 0. Często nazywany rzadkim (ang. sparse vector)

In [3]:
documents = [
    "Care About Your Craft",
    "Make Quality a Requirements Issue",
    "Don't Repeat Yourself",
    "Always Design for Concurrency",
    "Sign Your Work",
    "Refactor Early, Refactor Often",
]

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [5]:
cv = CountVectorizer(binary=True)
cv.fit(documents)
X = cv.transform(documents)

In [6]:
DF = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())
display(DF[:2], DF[4:6])

/Users/jkanclerz/myplace/dydaktyka/analiza-tekstu/.venv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,about,always,care,concurrency,craft,design,don,early,for,issue,make,often,quality,refactor,repeat,requirements,sign,work,your,yourself
0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0


,about,always,care,concurrency,craft,design,don,early,for,issue,make,often,quality,refactor,repeat,requirements,sign,work,your,yourself
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0
5,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0


Transformacja pozwoliła uzyskać strukturę w której każdy wiersz reprezentuje dokument, każda kolumna określa częstość wystąpienia danego tokenu

## Wektor częstości

In [ ]:
cv = CountVectorizer(binary=False)
cv.fit(documents)
X = cv.transform(documents)
DF = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())
display(DF[:2], DF[4:6])

,about,always,care,concurrency,craft,design,don,early,for,issue,make,often,quality,refactor,repeat,requirements,sign,work,your,yourself
0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0


,about,always,care,concurrency,craft,design,don,early,for,issue,make,often,quality,refactor,repeat,requirements,sign,work,your,yourself
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0
5,0,0,0,0,0,0,0,1,0,0,0,1,0,2,0,0,0,0,0,0


### zalety
+ Jest prostą numeryczną reprezentacją danych tekstowych.

### wady

- Problem stanowi rozmiar wektora który jest równy długości całego słownika.
- Nie uwzględnia relacji pomiędzy semantyką wyrazów
- relacje pomiędzy słowami - rózne interpretacje
- znaczenie – te same słowa różne znaczenie
- metafory, tautologie, ironie
- znaki specialne
- homonimy - to samo brzmienie inne znaczenia
- synonimy, idiomy
- 150k słów w słowniku


## Ważony wektor częstości
 

TF-IDF Term Frequency - Inverse Document Frequency pozwala uzyskać wartość określającą istotność danego tokenu w kontekście całego dokumentu. 
Wartość częstości (TF) nie uwzględnia różnicy w wartości informacyjnej jaką niosą ze sobą poszczególne wyrazy.
IDF - pozwala tą różnicę uchwycić.

Innymi słowy metodad TF-IDF pozwala wzmocnić znaczenie tokenu wraz ze wzrostem ilości wystąpień w dokumencie. Jednocześnie niwelując efekt wzmocnienia jeżeli dany token występuje równie często w pozostałych dokumentach stanowiących korpus.

Wartość tf-idf może być policzona przez mnożenie wartości tf oraz idf

$$ TfIdf(t,d) = tf(t,d)*idf(t,d)$$


$tf(t,d)$ jest częstością występowania tokenu w dokumencie
$idf(t,d)$ obliczany jest wg wzoru

$$ idf(t,d) = log\frac{n_d}{1+df(d,t)}$$
$n_d$ - liczba wystąpień w korpusie

$df(d,t)$ - liczba kokumentów gdzie występuje token t

$1$ - stała 1 jest opcjonalna, niemniej pozwala uzyskać niezerowe wartości dla tokenów występujących we wszystkich dokumentach


Implementacja TF-IDF w scikit-learn różni się jednak od powyższej definicji jest liczony następująco:
$$ TfIdf(t,d) = tf(t,d)*(idf(t,d) + 1)$$
$$ idf(t,d) = log\frac{1 + n_d}{1+df(d,t)}$$


In [ ]:
documents = [
    "Test Early. Test Often. Test Automatically!",
    "Refactor Early, Refactor Often, Just as you might!",
    "Abstractions Live Longer than Details",
    "Use Saboteurs to Test Your Testing",
    "Keep Knowledge in Plain Text",
]

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

tdif = TfidfVectorizer()
X = tdif.fit(documents)
X = tdif.transform(documents)
DF = pd.DataFrame(X.toarray(), columns=tdif.get_feature_names())
DF

,abstractions,as,automatically,details,early,in,just,keep,knowledge,live,longer,might,often,plain,refactor,saboteurs,test,testing,text,than,to,use,you,your
0,0.000000,0.000000,0.350068,0.000000,0.282433,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.282433,0.000000,0.000000,0.000000,0.847299,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.327881,0.000000,0.000000,0.264532,0.000000,0.327881,0.000000,0.000000,0.000000,0.000000,0.327881,0.264532,0.000000,0.655761,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.327881,0.000000
2,0.447214,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.447214,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.420669,0.339393,0.420669,0.000000,0.000000,0.420669,0.420669,0.000000,0.420669
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.447214,0.447214,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000


### zalety

* wielkość dokumentu nie ma wpływu wartość, tfifd która jest normalizowana
* uwzględnia różnice w znaczeniu tokenów dla znaczenia dokumentu

### wady

- Nie uwzględnia relacji pomiędzy semantyką wyrazów
- relacje pomiędzy słowami - rózne interpretacje
- znaczenie – te same słowa różne znaczenie
- homonimy - to samo brzmienie inne znaczenia
- ignoruje kolejność wyrazów w dokumencie

## Wektory - odległość

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cosine.html

### Podobieństwo wektorów / odległość kosinusowa

Jest metryką pozwalającą zmierzyć jak 2 wektory są podobne względem siebie. Wartość otrzymujemy obliczając wartość kosinusa konta pomiędzy wektorami w przestrzeni. Odległosć kosinusowa pozwala zniwelować wartości wynikające z położenia wektorów w przestrzeni, zachowując interpretowalną statystykę podobieństwa. Zakres wartości (0,1)



![cosine](http://blog.jkan.pl/images/cosine-distance.svg)

$ A = (3,3)$
$ B = (5,2)$
$ C = (5,0.5)$

$$Cos\theta = \frac{\vec{a} * \vec{b}}{||\vec{a}|| ||\vec{b}||} = \frac{\sum_1^n a_ib_i}{\sqrt{\sum_1^na_i^2} * \sqrt{\sum_1^nb_i^2}}$$
$$Cos\theta = \frac{15 + 6}{\sqrt{18} * \sqrt{29}}$$
$$Cos\theta = 0,9191$$

$$cos(90) = 0$$
$$cos(0) = 1$$
Kosunius przyjmuje wartość ``0`` dla wektorów prostopadłych (ortogonalnych) i wartość ``1`` dla wektorów równoległych. Im wartość kosinusa bliższa ``1`` tym mniejszy kont pomiędzy wektorami

Wykorzystując metodę cosine z ``scipy.spatial.distance`` musimy pamiętać o przekształceniu wg wzoru: 

$$Cosine Distance = 1 − Cosine Similarity$$

Oczekujemy że $podobieństwo(A,B) > podobieństwo(A,C)$ ponieważ kąt pomiędzy wektorami A,B jest mniejszy



In [ ]:
from scipy.spatial.distance import cosine
A = (3,3)
B = (5,2)
C = (5,0.5)
display(1 - cosine(A,B))
display(1 - cosine(A,C))

0.9191450300180578

0.7739572992033211

In [ ]:
A = (1,0,0)
B = (0,1,0)
1 - cosine(A,B)
0.0

0.0

In [ ]:
A = (1,0,0)
1 - cosine(A,A)
1.0

1.0

## Inne metryki
* Manhattan distance
* Euclidean distance
* Minkowski distance
* Jaccard similarity

In [ ]:
from scipy.spatial.distance import jaccard
from scipy.spatial.distance import minkowski
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cityblock
from scipy.spatial.distance import cosine

A = (3, 22, 55, 13)
B = (1, 12, 40, 5)

jaccard(A,B), minkowski(A,B), euclidean(A,B), cityblock(A,B), cosine(A,B)
(1.0, 19.82422760159901, 19.82422760159901, 35, 0.008847457489864041)

(1.0, 19.82422760159901, 19.82422760159901, 35, 0.008847457489864041)

## Reprezentacja wektorowa

In [ ]:
BOOK_LINES.head()

,author,content_txt,words
0,Mickiewicz,adam mickiewicz dziady,3
1,Mickiewicz,widowisko,1
2,Mickiewicz,część i / prawa strona teatru — dziewica w sam...,16
3,Mickiewicz,fortepiano,1
4,Mickiewicz,okno z lewej strony w pole; na prawej wielkie ...,19


In [ ]:
vectorizer = CountVectorizer() # Binary False
vectorizer.fit(train_df['content_txt'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
sample = train_df.iloc[120]['content_txt']

In [ ]:
sample

'tam i przyjaźni prawdziwej być nie może'

In [ ]:
vector = vectorizer.transform([sample])

In [ ]:
list(vector)

[<1x127720 sparse matrix of type '<class 'numpy.int64'>'
 	with 6 stored elements in Compressed Sparse Row format>]

In [ ]:
len(vectorizer.vocabulary_)

127720

In [ ]:
X_train = vectorizer.transform(train_df['content_txt'])
X_test = vectorizer.transform(test_df['content_txt'])
Y_train = train_df['author']
Y_test = test_df['author']

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(class_weight='balanced', dual=True)

Pierwszy z parametrów pomaga zrównoważyć nierównomierne ilości tekstów poszczególnych autorów przypisując im wagi odwrotnie proporcjonalne do częstotliwości występowania danej klasy. Drugi pozwala na wewnętrzne wykorzystanie innego sposobu implementacji algorytmu regresji logistycznej, który jest znacznie szybszy jeśli liczba cech przewyższa ilość próbek

In [ ]:
X_train.shape, Y_train.shape

((296289, 127720), (296289,))

In [ ]:
len(vectorizer.vocabulary_) > len(train_df)

False

In [ ]:

X_train

<296289x127720 sparse matrix of type '<class 'numpy.int64'>'
	with 1429080 stored elements in Compressed Sparse Row format>

In [ ]:

model = LogisticRegression(class_weight='balanced', dual=False, solver="liblinear")

In [ ]:

model

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

## Ocena modelu

In [ ]:
model.score(X_test, test_df['author'])

0.6492816135597339

Trzeba też pamiętać, że accuracy bardzo często nie jest dobrą miarą oceny jakości modelu. Bez wchodzenia w zbyt wiele detali nadmienię że podobnie jest w naszym przypadku. Jako przykład niech posłużą bardziej szczegółowe wyniki miar precision, recall (inaczej sensitivity, czułość), i F1 dla poszczególnych klas:

https://en.wikipedia.org/wiki/Precision_and_recall

https://en.wikipedia.org/wiki/Sensitivity_and_specificity

https://en.wikipedia.org/wiki/F1_score


In [ ]:
from sklearn import metrics
target = test_df['author']
predicted = model.predict(X_test)
print (metrics.classification_report(target, predicted, digits=4))

              precision    recall  f1-score   support

  Mickiewicz     0.1138    0.3219    0.1682       320
  Orzeszkowa     0.6379    0.6237    0.6307      7323
        Prus     0.6165    0.5621    0.5880      6959
     Reymont     0.6434    0.6987    0.6699      7756
 Sienkiewicz     0.7310    0.6981    0.7142     10563

    accuracy                         0.6493     32921
   macro avg     0.5485    0.5809    0.5542     32921
weighted avg     0.6594    0.6493    0.6532     32921



In [ ]:
BOOK_LINES.groupby('author').count()

,content_txt,words
author,,
Mickiewicz,3206,3206
Orzeszkowa,73230,73230
Prus,69588,69588
Reymont,77560,77560
Sienkiewicz,105626,105626


# Wykorzystanie innych modeli

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, Y_train)
model.score(X_test, test_df['author'])



0.6658971477172625

In [ ]:
from sklearn import metrics

target = test_df['author']
predicted = model.predict(X_test)
print (metrics.classification_report(target, predicted, digits=4))

              precision    recall  f1-score   support

  Mickiewicz     0.7500    0.0094    0.0185       320
  Orzeszkowa     0.6859    0.5954    0.6374      7323
        Prus     0.6317    0.5545    0.5906      6959
     Reymont     0.7125    0.6706    0.6909      7756
 Sienkiewicz     0.6463    0.8046    0.7168     10563

    accuracy                         0.6659     32921
   macro avg     0.6853    0.5269    0.5308     32921
weighted avg     0.6686    0.6659    0.6596     32921



In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

model = LinearSVC()
model.fit(X_train, Y_train)
model.score(X_test, test_df['author'])

target = test_df['author']
predicted = model.predict(X_test)
print (metrics.classification_report(target, predicted, digits=4))

              precision    recall  f1-score   support

  Mickiewicz     0.3864    0.1594    0.2257       320
  Orzeszkowa     0.6476    0.5918    0.6185      7323
        Prus     0.6138    0.5498    0.5800      6959
     Reymont     0.6767    0.6756    0.6761      7756
 Sienkiewicz     0.6630    0.7608    0.7085     10563

    accuracy                         0.6527     32921
   macro avg     0.5975    0.5475    0.5618     32921
weighted avg     0.6497    0.6527    0.6490     32921

